# 3. Generator Node

## 1) Role

In [1]:
from typing import List, TypedDict

class RAGState(TypedDict):
    question: str
    optimized_queries: List[str]

In [ ]:
# langchain_openai 가 없는 경우 설치 (한 번만 실행)
%pip install -q langchain-openai


In [2]:
# 구현 상세
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- [Prompt Engineering] ---
generator_prompt_template = """
당신은 기업 내부 문서를 기반으로 정확한 정보를 제공하는 AI 어시스턴트입니다.
아래의 [참고 문서]만을 사용하여 사용자의 [질문]에 답변하세요.

### 지침 사항
1. **사실 기반:** 오직 [참고 문서]에 있는 내용만 포함하세요. 외부 지식이나 당신의 상식을 섞지 마세요.
2. **인용 표기:** 답변의 각 문장이 어떤 문서에서 왔는지 대괄호 번호로 출처를 밝히세요. (예: ...라고 합니다. [1])
3. **모름 인정:** [참고 문서]에서 답을 찾을 수 없다면 솔직하게 "제공된 문서에서 관련 내용을 찾을 수 없습니다."라고 답하세요.
4. **명확성:** 전문 용어는 문서에 정의된 대로 사용하고, 요약하여 간결하게 작성하세요.

### 참고 문서 (Context)
{context}

### 질문 (Question)
{question}

### 답변:
"""

def format_docs(docs: List[str]) -> str:
    """문서 리스트를 번호가 매겨진 텍스트로 변환"""
    formatted = []
    for i, doc in enumerate(docs):
        formatted.append(f"[{i+1}] {doc}")
    return "\n\n".join(formatted)

def generate_node(state: RAGState) -> dict:
    print("--- [Step 3] Generator 시작 ---")
    
    question = state["question"]
    docs = state["retrieved_docs"]
    
    # 1. 문서 포맷팅 ([1] 문서내용... [2] 문서내용...)
    context_str = format_docs(docs)
    
    # 2. 모델 설정 (생성은 정교해야 하므로 gpt-4o 권장)
    llm = ChatOpenAI(model="gpt-4o", temperature=0) # 온도를 0으로 하여 창의성 억제
    
    prompt = ChatPromptTemplate.from_template(generator_prompt_template)
    
    # 3. 체인 실행
    chain = prompt | llm | StrOutputParser()
    
    response = chain.invoke({
        "question": question,
        "context": context_str
    })
    
    print(f"--- 생성 완료: {response[:50]}... ---")
    
    return {"final_answer": response}

/mnt/data1/work/sm-ai-v2/my-rag-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
